In [12]:
import torch
import torchvision
import torchvision.transforms as transforms
import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import make_grid
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
image_path = '/Users/patricknaylor/Desktop/Field_Detection/Images/Masked/'
torch.__version__
device = torch.device('mps')



In [13]:

images_paths = glob.glob(image_path + '*.jpg')
mask_paths = glob.glob(image_path + '*.npy')
scale_percent = 20

images = []
masks = []
for (img, msk) in zip(images_paths, mask_paths):
    image = cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2RGB)
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    images.append(cv2.resize(image, (width, height), interpolation = cv2.INTER_AREA))
    mask = np.load(msk).T
    mask = np.expand_dims(mask[25: -25, 25: -25], axis=2)
    mask = np.resize(mask, (72, 128, 1))
    masks.append(mask)


In [14]:
images_arr = np.stack(images)
masks_arr = np.stack(masks)


In [15]:
images_arr = images_arr.astype(np.uint8)
masks_arr = masks_arr.astype(np.uint8)

In [16]:
class imageDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        image = self.X[index]
        mask = self.y[index]
        y = torch.tensor(mask)
        X = self.transform(image)
        return [X, y]

    transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()])

    


    

In [17]:
print(np.shape(masks_arr))

(2753, 72, 128, 1)


In [18]:
batch_size = 8
transformed_dataset = imageDataset(X=images_arr, y=masks_arr)
train_set, val_set = torch.utils.data.random_split(transformed_dataset, [2000, 753])
train_dl = DataLoader(train_set, batch_size, shuffle=True)
val_dl = DataLoader(val_set, batch_size, shuffle=True)


In [19]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5, padding='same')
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(int(6*144*256*.25), int(2*144*256*.25))
        self.fc2 = nn.Linear(int(2*144*256*.25), int(144*256*.25))
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        return x


In [20]:
def cross_entropy(output, target):
    target = torch.flatten(target, 1)
    loss = -np.sum(target*np.log(output))
    return loss/float(output.shape[0])

model = Model().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [21]:
num_epochs = 32
for epoch in range(num_epochs):
    running_loss = 0.0
    sum = 0
    for i, (input, target) in enumerate(train_dl):
        input = input.to(device)
        target = torch.flatten(target, 1).to(device)

        optimizer.zero_grad()

        outputs = model(input)
        loss = criterion(outputs, target.float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        sum += 1

    print(f'[{epoch + 1}] loss: {running_loss / sum:.3f}')
    running_loss = 0.0


torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size([8, 9216])
torch.Size

KeyboardInterrupt: 